<a href="https://colab.research.google.com/github/hc2twv/UPSE_OP/blob/main/OptApreMachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Un ejemplo que combine optimización y aprendizaje automático para predecir la demanda de tráfico y luego optimizar la red de telecomunicaciones basada en esas predicciones. Utilizaremos scikit-learn para el aprendizaje automático y PuLP para la optimización.

Vamos a seguir estos pasos:

* Generar datos históricos de tráfico.
* Entrenar un modelo de aprendizaje automático para predecir futuras demandas de tráfico.
* Usar las predicciones para optimizar la red de telecomunicaciones.

Paso 1: Generar datos históricos de tráfico
Para este ejemplo, generaremos datos sintéticos de tráfico.

Paso 2: Entrenar un modelo de aprendizaje automático
Utilizaremos un modelo de regresión para predecir las futuras demandas de tráfico basadas en datos históricos.

Paso 3: Optimizar la red de telecomunicaciones
Usaremos las predicciones del modelo para optimizar la red utilizando PuLP.

In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 24.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pulp

# Paso 1: Generar datos históricos de tráfico
np.random.seed(0)
dates = pd.date_range('2023-01-01', periods=100, freq='D')
traffic_data = {
    'A_D': np.random.poisson(lam=20, size=100),
    'B_D': np.random.poisson(lam=30, size=100),
    'A_C': np.random.poisson(lam=15, size=100)
}
traffic_df = pd.DataFrame(traffic_data, index=dates)

# Paso 2: Entrenar un modelo de aprendizaje automático
# Crear características temporales
traffic_df['day_of_week'] = traffic_df.index.dayofweek
traffic_df['month'] = traffic_df.index.month

# Separar en características y etiquetas
X = traffic_df[['day_of_week', 'month']]
y_A_D = traffic_df['A_D']
y_B_D = traffic_df['B_D']
y_A_C = traffic_df['A_C']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_A_D_train, y_A_D_test = train_test_split(X, y_A_D, test_size=0.2, random_state=0)
X_train, X_test, y_B_D_train, y_B_D_test = train_test_split(X, y_B_D, test_size=0.2, random_state=0)
X_train, X_test, y_A_C_train, y_A_C_test = train_test_split(X, y_A_C, test_size=0.2, random_state=0)

# Entrenar los modelos de regresión
model_A_D = LinearRegression()
model_A_D.fit(X_train, y_A_D_train)

model_B_D = LinearRegression()
model_B_D.fit(X_train, y_B_D_train)

model_A_C = LinearRegression()
model_A_C.fit(X_train, y_A_C_train)

# Predecir futuras demandas
future_dates = pd.date_range('2024-01-01', periods=10, freq='D')
future_features = pd.DataFrame({
    'day_of_week': future_dates.dayofweek,
    'month': future_dates.month
})

future_A_D = model_A_D.predict(future_features)
future_B_D = model_B_D.predict(future_features)
future_A_C = model_A_C.predict(future_features)

# Usar la media de las predicciones para la optimización
predicted_demands = {
    ('A', 'D'): int(np.mean(future_A_D)),
    ('B', 'D'): int(np.mean(future_B_D)),
    ('A', 'C'): int(np.mean(future_A_C))
}

# Paso 3: Optimizar la red de telecomunicaciones usando PuLP
# Definir enlaces (de, a) con capacidad y costo
enlaces = {
    ('A', 'B'): {'capacidad': 25, 'costo': 2},
    ('A', 'C'): {'capacidad': 20, 'costo': 3},
    ('B', 'C'): {'capacidad': 30, 'costo': 1},
    ('B', 'D'): {'capacidad': 20, 'costo': 2},
    ('C', 'D'): {'capacidad': 35, 'costo': 1}
}

# Crear el problema de minimización
problem = pulp.LpProblem("Telecom_Network_Optimization", pulp.LpMinimize)

# Crear las variables de decisión
x = pulp.LpVariable.dicts("x", enlaces, lowBound=0, cat='Continuous')

# Definir la función objetivo (minimizar costos operativos y de transmisión)
problem += pulp.lpSum([x[(i, j)] * enlaces[(i, j)]['costo'] for (i, j) in enlaces]), "Costo_Total"

# Definir las restricciones de capacidad de los enlaces
for (i, j) in enlaces:
    problem += x[(i, j)] <= enlaces[(i, j)]['capacidad'], f"Capacidad_Enlace_{i}_{j}"

# Definir las restricciones de balance de flujo en cada nodo
nodos = ['A', 'B', 'C', 'D']
for nodo in nodos:
    flujo_entrante = pulp.lpSum([x[(i, j)] for (i, j) in enlaces if j == nodo])
    flujo_saliente = pulp.lpSum([x[(i, j)] for (i, j) in enlaces if i == nodo])

    demanda_entrante = pulp.lpSum([demanda for (origen, destino), demanda in predicted_demands.items() if destino == nodo])
    demanda_saliente = pulp.lpSum([demanda for (origen, destino), demanda in predicted_demands.items() if origen == nodo])

    problem += (flujo_entrante + demanda_saliente) == (flujo_saliente + demanda_entrante), f"Balance_Flujo_{nodo}"

# Resolver el problema
problem.solve()

# Imprimir los resultados
print("Estado:", pulp.LpStatus[problem.status])
print("Costo Total:", pulp.value(problem.objective))
for (i, j) in enlaces:
    print(f"Flujo en el enlace {i}->{j}: {pulp.value(x[(i, j)])} unidades")


Estado: Optimal
Costo Total: 182.0
Flujo en el enlace A->B: 14.0 unidades
Flujo en el enlace A->C: 20.0 unidades
Flujo en el enlace B->C: 29.0 unidades
Flujo en el enlace B->D: 15.0 unidades
Flujo en el enlace C->D: 35.0 unidades


Explicación del código
* Generación de datos históricos: Se generan datos sintéticos de tráfico para tres enlaces principales (A_D, B_D, A_C) utilizando una distribución de Poisson para simular tráfico diario durante 100 días.

* Entrenamiento de modelos de regresión: Se crean características temporales (día de la semana y mes) y se entrenan modelos de regresión lineal para predecir las futuras demandas de tráfico en los tres enlaces.

* Predicción de futuras demandas: Se utilizan los modelos de regresión para predecir las demandas de tráfico para los próximos 10 días y se calcula la media de las predicciones para usarlas en la optimización.

* Optimización de la red de telecomunicaciones: Se define y resuelve un problema de optimización utilizando PuLP para minimizar los costos operativos y de transmisión mientras se satisfacen las demandas de tráfico predichas.